In [14]:
%%writefile app.py
import pickle
import re
from io import BytesIO

import PyPDF2
import numpy as np
import requests
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the model and components
model = load_model('./sequence_2_sequence.keras')
encoder_model = load_model('encoder_model.keras')
decoder_model = load_model('decoder_model.keras')

with open('model_components.pkl', 'rb') as f:
    components = pickle.load(f)

x_tokenizer = components['x_tokenizer']
reverse_target_word_index = components['reverse_target_word_index']
target_word_index = components['target_word_index']
max_text_len = components['max_text_len']
max_summary_len = components['max_summary_len']


# Function to generate summary
def generate_summary(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))

    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:

        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if sampled_token != 'eostok':
            decoded_sentence += ' ' + sampled_token

        # Exit condition: either hit max length or find stop word.
        if sampled_token == 'eostok' or len(decoded_sentence.split()) >= (max_summary_len - 1):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence


# Function to extract text from PDF
def extract_text_from_pdf(pdf_file):
    pdf_reader = PyPDF2.PdfReader(BytesIO(pdf_file.read()))
    c = 0
    text = ""

    while c < len(pdf_reader.pages):
        pageObj = pdf_reader.pages[c]
        # text += pageObj.extract_text()
        page_text = pageObj.extract_text()
        if page_text:
            # Add line breaks based on common patterns
            page_text = re.sub(r'\n+', '\n', page_text)
            text += page_text + "\n"
        c += 1

    return text.replace('\n', ' ')


# Title of the app
st.title("Text Summarization App")

# Option to upload PDF or enter text
option = st.radio("Choose input method", ["Upload PDF", "Enter Text"])

# Handle PDF upload
if option == "Upload PDF":
    uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")
    if uploaded_file is not None:
        # Extract text from PDF
        pdf_text = extract_text_from_pdf(uploaded_file)
        st.text_area("Extracted Text", pdf_text, height=130, )
        text_to_process = pdf_text
    else:
        text_to_process = ""

# Handle direct text input
elif option == "Enter Text":
    text_to_process = st.text_area("Enter your paragraph to summarize here:", height=100)

# Button to process the text
if st.button("Summarize Text"):
    if text_to_process:
        response = requests.post(
            'http://127.0.0.1:5000/process',
            json={'paragraph': text_to_process}
        )

        if response.status_code == 200:
            result = response.json()
            st.subheader("Extractive Summary:")
            st.write(result.get('summary'))
        else:
            st.error(f"Error: {response.json().get('error')}")

        # Preprocess the input
        input_seq = x_tokenizer.texts_to_sequences([text_to_process])
        input_seq = pad_sequences(input_seq, maxlen=max_text_len, padding='post')

        # Generate summary
        abstractive_summary = generate_summary(input_seq)

        st.subheader("Abstractive Text")
        st.write(abstractive_summary)
    else:
        st.error("Please enter some text before processing.")


Overwriting app.py


In [ ]:
import os

file_path = 'sequence_2_sequence.keras'
print(os.path.isfile(file_path))  # Should return True if the file exists

In [ ]:
from tensorflow.keras.models import load_model

model_path = 'sequence_2_sequence.keras'
model = load_model(model_path)


In [ ]:
!chmod +r sequence_2_sequence.keras

In [ ]:
# Read the first few bytes of the file to confirm its presence
with open('sequence_2_sequence.keras', 'rb') as file:
    print(file.read(100))  # Read first 100 bytes


In [ ]:
from tensorflow.keras.models import load_model

model_path = 'sequence_2_sequence.keras'
model = load_model(model_path)


In [ ]:
import os
from tensorflow.keras.models import load_model

# Get the absolute path to the file
model_path = os.path.abspath('sequence_2_sequence.h5')
model = load_model(model_path)


In [ ]:
import os

print("Current Working Directory:", os.getcwd())
print("Files in directory:", os.listdir())

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# 
# # Create a simple model
# simple_model = Sequential([
#     Dense(10, input_shape=(5,), activation='relu'),
#     Dense(1)
# ])
# 
# # Save and load the model
# simple_model.save('test_model.keras')
# loaded_model = load_model('test_model.keras')


In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# 
# # Create a simple model
# model = Sequential([
#     Dense(10, input_shape=(5,), activation='relu'),
#     Dense(1)
# ])
# 
# # Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error')
# 
# # Save the model to an HDF5 file
# model.save('model.h5')


In [ ]:
# from tensorflow.keras.models import load_model
# 
# # Load the model from the HDF5 file
# model = load_model('model.h5')
# 
# # Check the model summary
# model.summary()

In [ ]:
# model.compile_metrics()

In [ ]:
import os

base_path = os.getcwd()
model = load_model(os.path.join(base_path, 'sequence_2_sequence.h5'))
encoder_model = load_model(os.path.join(base_path, 'encoder_model.h5'))
decoder_model = load_model(os.path.join(base_path, 'decoder_model.h5'))

with open(os.path.join(base_path, 'model_components.pkl'), 'rb') as f:
    components = pickle.load(f)

In [ ]:
import os

file_path = 'sequence_2_sequence.keras'
if os.path.exists(file_path):
    print('exist.')
    model = load_model(file_path)
else:
    print(f"File not found: {file_path}")

In [ ]:
try:
    model = load_model('sequence_2_sequence.keras')
except Exception as e:
    print(f"Error loading model: {e}")

In [11]:
# Saving
# model.save('sequence_2_sequence.h5')

# Loading
from tensorflow.keras.models import load_model

model = load_model('sequence_2_sequence.h5')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 200)  │  6,680,400 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 100,      │    601,200 │ embedding[0][0]   │
│                     │ 300), (None,      │            │                   │
│                     │ 300), (None,      │            │                   │
│                     │ 300)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 100,      │    721,200 │ lstm[0][0]        │
│                     │ 300), (None,      │            │                   │
│                     │ 300), (None,      │            │                   │
│                     │ 300)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 200) │  2,317,200 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 100,      │    721,200 │ lstm_1[0][0]      │
│                     │ 300), (None,      │            │                   │
│                     │ 300), (None,      │            │                   │
│                     │ 300)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, None,     │    601,200 │ embedding_1[0][0… │
│                     │ 300), (None,      │            │ lstm_2[0][1],     │
│                     │ 300), (None,      │            │ lstm_2[0][2]      │
│                     │ 300)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, None,      │  3,487,386 │ lstm_3[0][0]      │
│ (TimeDistributed)   │ 11586)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,129,788 (57.72 MB)

 Trainable params: 15,129,786 (57.72 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [10]:
with open('sequence_2_sequence.keras', 'rb') as f:
    content = f.read()
    print(f"File size: {len(content)} bytes")

File size: 121093677 bytes


In [1]:
model

NameError: name 'model' is not defined

In [9]:
model = load_model('./sequence_2_sequence.h5')

In [8]:
encoder_model = load_model('encoder_model.h5')

In [7]:
decoder_model = load_model('decoder_model.keras')

ValueError: File not found: filepath=decoder_model.keras. Please ensure the file is an accessible `.keras` zip file.